<a href="https://colab.research.google.com/github/gverafei/artificial-networks-technologies/blob/main/tarea7/model/tarea7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tarea 7: Aplicación de CNN en Visión Artificial**

**Instrucciones:**

Aplique una CNN preentrenada, tales como VGGNet, ResNet, MobileNet, YOLO o cualquier otra que haya investigado para resolver un caso práctico de visión por computadora, utilizando técnicas de carga de modelos, extracción de características y fine-tuning.

El problema a resolver es libre.

Para este caso, se utilizara VGG-Net16 preentrenada para clasificar perros y gatos en tiempo real.

**Entregables**

+ Documento del trabajo (Springer)
+ Código de la implementación (.zip o link al repositorio en Github)


## A. Configure virtual environment

Estas línea solo se ejecutan la primera vez. Solicitará crear un virtual environment. En un notebook en línea como Colab, pedirá selecciona el kernel de la esquina superior derecha.

In [ ]:
!python3 -m venv .venv

In [ ]:
!source .venv/bin/activate # Linux/Mac
# !.\venv\Scripts\activate # Windows

In [ ]:
!pip install --upgrade pip --quiet

## B. Instala dependencias necesarias

In [ ]:
!pip install numpy --quiet

In [ ]:
!pip install matplotlib --quiet

In [ ]:
!pip install tensorflow --quiet

In [ ]:
pip install tensorflow-datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install opencv-python --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install scikit-learn --quiet

In [ ]:
!pip install pandas --quiet

In [ ]:
!pip install --upgrade certifi --quiet

## C. Configura el ambiente de pandas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 3)
pd.set_option('display.width', 800)

print("Versiones -> pandas:", pd.__version__)

Versiones -> pandas: 2.2.2



## 1. Cargamos el dataset a utilizr


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Descargar el set de datos de perros y gatos
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.VYALGQ_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
#Imprimir los metadatos para revisarlos
metadatos

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/Users/gvera/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douc

In [ ]:
# Opcional: cache + prefetch para acelerar
AUTOTUNE = tf.data.AUTOTUNE
datos = datos.prefetch(buffer_size=AUTOTUNE)

#Una forma de mostrar 5 ejemplos del set
tfds.as_dataframe(datos['train'].take(5), metadatos)

: 

: 

In [ ]:
#Otra forma de mostrar ejemplos del set
tfds.show_examples(datos['train'], metadatos)

: 

: 

In [ ]:
#Manipular y visualizar el set
#Lo pasamos a TAMANO_IMG (224x224) que es el tamaño que requiere VGG16 y a blanco y negro (solo para visualizar)
import matplotlib.pyplot as plt
import cv2

plt.figure(figsize=(20,20))

# 1. Parámetros básicos
TAMANO_IMG=224      # VGG16 usa 224x224

for i, (imagen, etiqueta) in enumerate(datos['train'].take(25)):
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(imagen, cmap='gray')

In [ ]:
#Variable que contendra todos los pares de los datos (imagen y etiqueta) ya modificados (blanco y negro, 100x100)
datos_entrenamiento = []

In [ ]:
for i, (imagen, etiqueta) in enumerate(datos['train']): #Todos los datos
  imagen = cv2.resize(imagen.numpy(), (TAMANO_IMG, TAMANO_IMG))
  imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen = imagen.reshape(TAMANO_IMG, TAMANO_IMG, 1) #Cambiar tamano a 100,100,1
  datos_entrenamiento.append([imagen, etiqueta])